<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/AA-Stagecoach-Project/blob/main/File_share_16_Fetching_the_timetable_for_given_day_(automated).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Fetching data
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Fetch day of the week
    day_of_week = get_day_of_week()
    print(f"\nToday is {day_of_week}.")

    # Print extracted schedule names
    print("\nExtracted Schedule Names:")
    for i, schedule_name in enumerate(schedule_names_dict.keys(), start=1):
        print(f"{i}. {schedule_name}")

    # Determine the matching schedule name based on the current day of the week
    selected_schedule_name = None

    if day_of_week.lower() == 'monday' or day_of_week.lower() == 'tuesday' or day_of_week.lower() == 'wednesday' or day_of_week.lower() == 'thursday':
        # Check for the preferred schedule names for Monday to Thursday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    elif day_of_week.lower() == 'friday':
        # Check for the preferred schedule names for Friday
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Friday', 'Friday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    elif day_of_week.lower() == 'saturday':
        # Check for the preferred schedule names for Saturday
        preferred_schedule_names = ['Saturday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    elif day_of_week.lower() == 'sunday':
        # Check for the preferred schedule names for Sunday
        preferred_schedule_names = ['Sunday']
        for preferred_name in preferred_schedule_names:
            if preferred_name in schedule_names_dict:
                selected_schedule_name = preferred_name
                break
    else:
        # For other days of the week, select based on exact match
        for schedule_name in schedule_names_dict.keys():
            if day_of_week.lower() in schedule_name.lower():
                selected_schedule_name = schedule_name
                break

    if selected_schedule_name:
        print(f"\nSelected schedule name: {selected_schedule_name}")

        # Categorizing the timetable into hourly slots
        timetable = schedule_names_dict[selected_schedule_name]
        slots = categorize_into_slots(timetable)

        # Get the current time in BST
        bst = pytz.timezone('Europe/London')
        now = datetime.now(bst)
        current_hour = now.hour
        current_minute = now.minute

        # Calculate the number of buses scheduled between the first time available in the current slot and the current time
        bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
        print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

        # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
        hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
        print("\nTimetable:")
        print("Time   Line ID  Stop Point ID  Direction")
        for hour in hours_to_print:
            if 0 <= hour < 24:
                print(f"\nSlot {hour}:")
                for journey in slots[hour]:
                    journey_hour = str(journey['hour']).zfill(2)
                    journey_minute = str(journey['minute']).zfill(2)
                    time = f"{journey_hour}:{journey_minute}"
                    print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

    else:
        print("No matching schedule name found for today.")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: 22
Enter the stop point ID: 490012375E
Enter the direction (outbound/inbound): outbound

Today is Monday.

Extracted Schedule Names:
1. Friday
2. Monday to Thursday
3. Saturday
4. Sunday

Selected schedule name: Monday to Thursday

Number of buses scheduled between the first time available in the current slot and now: 4

Timetable:
Time   Line ID  Stop Point ID  Direction

Slot 12:
12:09   22    490012375E  Outbound
12:20   22    490012375E  Outbound
12:31   22    490012375E  Outbound
12:42   22    490012375E  Outbound
12:53   22    490012375E  Outbound

Slot 13:
13:05   22    490012375E  Outbound
13:16   22    490012375E  Outbound
13:28   22    490012375E  Outbound
13:39   22    490012375E  Outbound
13:51   22    490012375E  Outbound

Slot 14:
14:02   22    490012375E  Outbound
14:14   22    490012375E  Outbound
14:25   22    490012375E  Outbound
14:36   22    490012375E  Outbound
14:47   22    490012375E  Outbound
14:57   22    490012375E  Outbound
